# Creator: Ivan Bardarov <br> (University of Strathclyde, March 2019)
## This module deals with the collection of gram matrices, adds new items to the collection

In [1]:
import argparse
import os
import shutil
import sys
sys.path.append('../recommender/')
import numpy as np
from os import listdir, makedirs, remove
from collections import defaultdict
from os.path import isfile, join, exists
from cloth_detector import *
import gram_extractor as ge
import background_remover as br
from ntpath import basename
from tqdm import tqdm
from shutil import move

def get_files_from_path(image_path):
    """
    A function which only keeps the files in a folder
    
    Parameters
    ----------
    image_path : Path
        The path to the folder with images
        
    Returns
    -------
    list
        The paths of the files in the specified folder
        
    """
    return [join(image_path, f) for f in listdir(image_path) if isfile(join(image_path, f))]

def get_and_append(arr, filename):
    """
    A function which loads a stored collection and extends it
    
    Parameters
    ----------
    arr : list
        The list with gram matrices to be added
    filename: str
        The name of the file with the collection to be extended
        
    """
    stored_arr = np.load(filename)
    new_arr = np.concatenate((stored_arr, arr), axis=0)
    np.save(filename, new_arr)

def get_grams_collection(cat):
    """
    Returns the the grams collection from the file system
    
    Parameters
    ----------
    cat : int
        The id of the category for which we want to retrive the collection
        
    Returns
    -------
    ndarray
        An array with the gram matrices collection
    """
    try:
        return np.load(get_grams_filename(cat))
    except:
        print("No files in the database!")
        
def get_filenames_collection(cat):
    """
    Returns the the filenames collection from the file system
    
    Parameters
    ----------
    cat : int
        The id of the category for which we want to retrive the collection
        
    Returns
    -------
    ndarray
        An array with the filenames collection
    """
    try:
        return np.load(get_files_filename(cat))
    except:
        print("No files in the database!")

def get_grams_filename(cat):
    """
    Returns the name of the collection file for the gram matrices
    
    Parameters
    ----------
    cat : int
        The id of the category for which we want to retrive the collection
        
    Returns
    -------
    str
        the path to the collection
    """
    return f"../collections/{cat}_grams.npy"

def get_files_filename(cat):
    """
    Returns the name of the collection file for the file names
    
    Parameters
    ----------
    cat : int
        The id of the category for which we want to retrive the collection
        
    Returns
    -------
    str
        the path to the collection
    """
    return f"../collections/{cat}_filenames.npy"

if __name__ == "__main__":
    print("Image Processing...")
    # add required parameters to the script
    ap = argparse.ArgumentParser()
    ap.add_argument("-p", "--path", required=True, help="Path to the folder with images")
    args = vars(ap.parse_args())
    images_path = args["path"]
    
    if exists(images_path) and exists(images_path2):
        #make sure the destinations exist
        makedirs("../collections", exist_ok=True)
        makedirs("../static/image_data", exist_ok=True)
        temp = "../collections/temp.jpg"
        
        #cleanup the file paths
        files = get_files_from_path(images_path)
        
        #load the object detection model
        learner, id2cat, cat2id = load_learner()
        grams = defaultdict(list)
        filenames = defaultdict(list)
        for fn in tqdm(files):
            
            #make a prediction for each image
            bboxes, cats, _ = predict_fn(learner, fn)
            img = Image.open(fn)
            h, w = img.size
            
            #copy the file to the destination folder and preserve the name
            bn = basename(fn)
            shutil.copy(fn, f"../static/image_data/{bn}")
            
            # process all the clothing items in the photo
            for cat, bbox in zip(cats,bboxes):
                
                #get the bounding box and crop the image
                bbox = np.array([bbox[1]*h, bbox[0]*w, bbox[3]*h, bbox[2]*w])
                img.crop(tuple(bbox)).save(temp)
                
                #remove the background
                br.remove_background(temp).save(temp)
                
                #get the gram matrix
                features_vector = ge.get_gram(temp)
                
                #add to the local collection
                grams[cat].append(features_vector)
                filenames[cat].append(f"../static/image_data/{bn}")
            
            if exists(temp): remove(temp)
            
        #for each of the categories which have been extended alter the collection files
        for cat in grams.keys():
            grams_path = get_grams_filename(cat)
            fns_path = get_files_filename(cat)
            if exists(grams_path) and exists(fns_path):
                get_and_append(grams[cat], grams_path)
                get_and_append(filenames[cat], fns_path)
                continue
            np.save(grams_path, grams[cat])
            np.save(fns_path, filenames[cat])
    else: print("No such directory. Make sure it exists")

Image Processing...


100%|██████████| 180/180 [37:55<00:00, 12.43s/it]
